# Import dependencies and prepare default directory

In [1]:
import os
#try:
import supramolsim
#except:
#  !pip install git+https://github.com/HenriquesLab/SupraMolecularSimulator.git

from supramolsim.utils import data_format
from supramolsim.generate import coordinates_field as field
import supramolsim.workflows as workflows

pck_dir = os.path.dirname(os.path.abspath(supramolsim.__file__))
local_dir = os.path.join(pck_dir, "configs")
configuration_path = []
configuration_path.append(local_dir)

# Load scructure

In [ ]:
# NPC example
structure_id = "1XI5"
structure, structure_param = supramolsim.load_structure(
    structure_id, configuration_path[0]
)

In [ ]:
structure.show_assembly_atoms(view_init=[90,0,0], assembly_fraction=0.2)

# Add lables

In [ ]:
label_id = "Mock_primary_antibody"
target_info = dict(
    type="Sequence",
    value="EQATETQ"
)
fluorophore_id = "AF647"
lab_eff = 0.7
tmp_label1 = data_format.structural_format.label_builder_format(
    label_id, fluorophore_id, lab_eff, target_info
)
particle, label_params_list = supramolsim.particle_from_structure(
    structure, [tmp_label1], configuration_path[0]
)
particle.show_instance()

# (Optional) Add defects to the particle

In [5]:
## add defects
#short_dist_cluster = 300
#long_dist_cluster = 600
#deg_dissasembly = 0.2

#particle.add_defects(
#    eps1=short_dist_cluster,
#    xmer_neigh_distance=long_dist_cluster,
#    deg_dissasembly=deg_dissasembly,
#)

#particle.show_instance()

# Create particle field

In [ ]:
nparticles = 10
min_distance = 150
random = True
molecule_parameters = dict(minimal_distance=min_distance)
coordinates_field = field.create_min_field(nparticles=nparticles,
                                            random_placing=random,
                                            molecule_pars=molecule_parameters)
coordinates_field.create_molecules_from_InstanceObject(particle)
coordinates_field.construct_static_field()
exported_field = coordinates_field.export_field()

In [ ]:
coordinates_field.show_field()

# Create imager

In [ ]:
selected_mods = ["STED", "Confocal"]
imaging_system, modality_parameters = workflows.create_imaging_system(
    exported_field, selected_mods, configuration_path[0]
)

In [9]:
from supramolsim.utils.data_format import configuration_format

acq_params_per_mod = dict()
acq_params_per_mod["STED"] = configuration_format.format_modality_acquisition_params(
    exp_time=0.0001,
    nframes=20,
    save=False 
)
acq_params_per_mod["Confocal"] = configuration_format.format_modality_acquisition_params(
    exp_time=0.0001,
    nframes=10,
    save=False
)


In [ ]:
exp_name = "SupraMolSim_output_script"
# sav_dir = "YOUR_PATH"

output = workflows.generate_multi_imaging_modalities(
        image_generator=imaging_system,
        experiment_name=exp_name,
        acquisition_param=acq_params_per_mod,
)

In [11]:
saving_directory = None # change to your desired saving path
data_format.configuration_format.write_parameters(
    structure=structure_param,
    probes=label_params_list,
    virtualsample=exported_field,
    modalities=modality_parameters,
    acquisition=acq_params_per_mod,
    writingpath=saving_directory
)

# Preview your images

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2)
nframe = 0
axs[0].imshow(output['STED'][nframe])
axs[1].imshow(output['Confocal'][nframe])
